# SOLUCIONES - SESIÓN 3: Seguridad y Autenticación

**Contenido:** Soluciones completas y comentadas de los 10 ejercicios sobre OAuth2, JWT y bcrypt.

**Nota:** Revisa las soluciones solo después de intentar resolver los ejercicios por tu cuenta.

## CONFIGURACIÓN DEL ENTORNO

In [ ]:
# Instalación de dependencias
!pip install fastapi==0.115.0 uvicorn[standard]==0.32.0 passlib[bcrypt]==1.7.4 PyJWT==2.8.0 python-multipart==0.0.6 -q
print("✅ Dependencias instaladas")

In [ ]:
# Imports necesarios
from fastapi import FastAPI, Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from fastapi.testclient import TestClient
from passlib.context import CryptContext
import jwt
from jwt.exceptions import PyJWTError
from datetime import datetime, timedelta
from typing import Annotated, Optional
from pydantic import BaseModel

print("✅ Imports completados")

---

## SOLUCIÓN EJERCICIO 1: Hash de password con bcrypt

In [ ]:
# Crear contexto de hashing con bcrypt
# schemes=["bcrypt"] indica que usaremos el algoritmo bcrypt
# deprecated="auto" permite migrar automáticamente de esquemas antiguos
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

# Contraseña original
password_original = "mi_password_seguro"

# Hashear la contraseña
# El hash es IRREVERSIBLE: no se puede obtener la password original del hash
password_hash = pwd_context.hash(password_original)

# Verificación
assert pwd_context is not None
assert password_hash is not None
assert password_hash != password_original
assert len(password_hash) > 50
print("✅ Ejercicio 1 completado")
print(f"Hash generado: {password_hash}")
print(f"\nCaracterísticas del hash:")
print(f"  - Longitud: {len(password_hash)} caracteres")
print(f"  - Prefijo $2b$: Indica bcrypt")
print(f"  - Incluye salt automático (cada hash es único)")

---

## SOLUCIÓN EJERCICIO 2: Verificar password contra hash

In [ ]:
# Hash de ejemplo (password: "secret123")
hash_guardado = pwd_context.hash("secret123")

# Verificar password correcta
# pwd_context.verify() hashea el intento y lo compara con el hash guardado
# Retorna True si coinciden, False si no
password_correcta = pwd_context.verify("secret123", hash_guardado)

# Verificar password incorrecta
password_incorrecta = pwd_context.verify("otra_password", hash_guardado)

# Verificación
assert password_correcta == True
assert password_incorrecta == False
print("✅ Ejercicio 2 completado")
print(f"Password correcta válida: {password_correcta}")
print(f"Password incorrecta rechazada: {password_incorrecta}")
print("\n💡 Nota: verify() NO decodifica el hash, lo compara matemáticamente")

---

## SOLUCIÓN EJERCICIO 3: Generar JWT con payload personalizado

In [ ]:
# Configuración JWT
SECRET_KEY = "clave-secreta-ejercicio"
ALGORITHM = "HS256"

# Crear payload con claims personalizados
# "sub" (subject): Identificador del usuario (estándar JWT)
# "email" y "role": Claims personalizados
# "exp" (expiration): Timestamp de expiración (estándar JWT)
payload = {
    "sub": "usuario123",
    "email": "usuario@example.com",
    "role": "admin",
    "exp": datetime.utcnow() + timedelta(minutes=30)
}

# Generar token JWT
# jwt.encode() toma el payload, lo firma con SECRET_KEY y retorna el token
token = jwt.encode(payload, SECRET_KEY, algorithm=ALGORITHM)

# Verificación
assert "sub" in payload
assert "email" in payload
assert "exp" in payload
assert token is not None
assert isinstance(token, str)
print("✅ Ejercicio 3 completado")
print(f"Token generado: {token[:50]}...")
print(f"\nEstructura del token:")
partes = token.split('.')
print(f"  - Header: {partes[0][:20]}...")
print(f"  - Payload: {partes[1][:20]}...")
print(f"  - Signature: {partes[2][:20]}...")

---

## SOLUCIÓN EJERCICIO 4: Decodificar y validar JWT

In [ ]:
# Token de prueba (generado en ejercicio anterior)
token_valido = jwt.encode(
    {"sub": "testuser", "exp": datetime.utcnow() + timedelta(minutes=5)},
    SECRET_KEY,
    algorithm=ALGORITHM
)

# Decodificar token válido
# jwt.decode() verifica la firma y retorna el payload
# Si la firma no es válida o el token expiró, lanza PyJWTError
try:
    payload_decodificado = jwt.decode(
        token_valido, 
        SECRET_KEY, 
        algorithms=[ALGORITHM]
    )
    usuario = payload_decodificado.get("sub")
    print(f"✅ Token válido. Usuario: {usuario}")
except PyJWTError:
    print("❌ Token inválido")

# Intentar decodificar token FALSO
token_falso = "token.falso.aqui"
try:
    jwt.decode(token_falso, SECRET_KEY, algorithms=[ALGORITHM])
    token_falso_rechazado = False
except PyJWTError:
    # El token falso no tiene una firma válida, PyJWT lanza error
    token_falso_rechazado = True
    print("✅ Token falso correctamente rechazado")

# Verificación
assert payload_decodificado is not None
assert usuario == "testuser"
assert token_falso_rechazado == True
print("\n✅ Ejercicio 4 completado")
print("\n💡 Nota: PyJWT verifica automáticamente:")
print("  - Firma válida")
print("  - Token no expirado")
print("  - Formato correcto")

---

## SOLUCIÓN EJERCICIO 5: Endpoint POST /token (login)

In [ ]:
app = FastAPI()

# Base de datos fake
USERS_DB = {
    "admin": {
        "username": "admin",
        "email": "admin@example.com",
        "hashed_password": pwd_context.hash("admin123")
    }
}

# Modelo de respuesta
class Token(BaseModel):
    access_token: str
    token_type: str

# Endpoint de login
@app.post("/token", response_model=Token)
async def login(form_data: OAuth2PasswordRequestForm = Depends()):
    # 1. Buscar usuario en la base de datos
    user = USERS_DB.get(form_data.username)
    
    # 2. Validar que el usuario existe
    if not user:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Usuario o contraseña incorrectos",
            headers={"WWW-Authenticate": "Bearer"},
        )
    
    # 3. Verificar la contraseña
    if not pwd_context.verify(form_data.password, user["hashed_password"]):
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Usuario o contraseña incorrectos",
            headers={"WWW-Authenticate": "Bearer"},
        )
    
    # 4. Generar token JWT
    payload = {
        "sub": user["username"],
        "exp": datetime.utcnow() + timedelta(minutes=30)
    }
    token = jwt.encode(payload, SECRET_KEY, algorithm=ALGORITHM)
    
    # 5. Retornar el token
    return {"access_token": token, "token_type": "bearer"}

# Pruebas
client = TestClient(app)

# Login exitoso
response = client.post("/token", data={"username": "admin", "password": "admin123"})
assert response.status_code == 200
assert "access_token" in response.json()
print("✅ Login exitoso")
print(f"   Token: {response.json()['access_token'][:40]}...")

# Login fallido
response_fail = client.post("/token", data={"username": "admin", "password": "incorrecta"})
assert response_fail.status_code == 401
print("✅ Password incorrecta rechazada")

print("\n✅ Ejercicio 5 completado")
print("\n💡 Nota: OAuth2PasswordRequestForm espera formulario, NO JSON")

---

## SOLUCIÓN EJERCICIO 6: OAuth2PasswordBearer básico

In [ ]:
# Configurar OAuth2PasswordBearer
# tokenUrl="token" indica que el endpoint de login es POST /token
# Esta dependencia extrae el token del header: Authorization: Bearer <token>
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

# Endpoint protegido que usa oauth2_scheme
# Depends(oauth2_scheme) extrae automáticamente el token del header
# Si no hay header Authorization, retorna 401 automáticamente
@app.get("/protegido")
async def ruta_protegida(token: str = Depends(oauth2_scheme)):
    return {"mensaje": "Acceso autorizado", "token": token[:20]}

# Pruebas
# Login primero para obtener token
response_login = client.post("/token", data={"username": "admin", "password": "admin123"})
token = response_login.json()["access_token"]

# Acceso sin token
response_no_token = client.get("/protegido")
assert response_no_token.status_code == 401
print("✅ Acceso sin token rechazado")

# Acceso con token
response_with_token = client.get("/protegido", headers={"Authorization": f"Bearer {token}"})
assert response_with_token.status_code == 200
print("✅ Acceso con token autorizado")
print(f"   Respuesta: {response_with_token.json()}")

print("\n✅ Ejercicio 6 completado")
print("\n💡 Nota: oauth2_scheme solo EXTRAE el token, no lo valida")

---

## SOLUCIÓN EJERCICIO 7: Dependencia obtener_usuario_actual()

In [ ]:
# Modelo de usuario
class User(BaseModel):
    username: str
    email: str

# Dependencia que valida el token y retorna el usuario
async def obtener_usuario_actual(token: str = Depends(oauth2_scheme)) -> User:
    """
    Esta dependencia:
    1. Recibe el token (extraído por oauth2_scheme)
    2. Lo decodifica y valida
    3. Busca al usuario en la BD
    4. Retorna el objeto User o lanza 401
    """
    try:
        # 1. Decodificar token
        # jwt.decode() valida la firma y la expiración
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        
        # 2. Extraer username del payload
        # El claim "sub" (subject) contiene el identificador del usuario
        username: str = payload.get("sub")
        
        # 3. Buscar usuario en la base de datos
        user_dict = USERS_DB.get(username)
        
        # 4. Validar que el usuario existe
        if user_dict is None:
            raise HTTPException(
                status_code=status.HTTP_401_UNAUTHORIZED,
                detail="Usuario no encontrado",
                headers={"WWW-Authenticate": "Bearer"},
            )
        
        # 5. Retornar objeto User (sin la password)
        return User(
            username=user_dict["username"],
            email=user_dict["email"]
        )
    except PyJWTError:
        # Si el token es inválido o expiró, PyJWT lanza error
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Token inválido o expirado",
            headers={"WWW-Authenticate": "Bearer"},
        )

# Tipo reutilizable con Annotated
# Esto permite usar "current_user: CurrentUser" en vez de escribir toda la dependencia
CurrentUser = Annotated[User, Depends(obtener_usuario_actual)]

# Endpoint de prueba
@app.get("/test-user", response_model=User)
async def test_user(current_user: CurrentUser):
    return current_user

# Pruebas
response_user = client.get("/test-user", headers={"Authorization": f"Bearer {token}"})
assert response_user.status_code == 200
assert response_user.json()["username"] == "admin"
print("✅ Usuario extraído correctamente del token")
print(f"   Usuario: {response_user.json()}")

print("\n✅ Ejercicio 7 completado")
print("\n💡 Nota: Esta dependencia se puede reutilizar en TODOS los endpoints protegidos")

---

## SOLUCIÓN EJERCICIO 8: Endpoint protegido GET /usuarios/me

In [ ]:
# Endpoint que retorna el perfil del usuario autenticado
# Al usar CurrentUser como tipo, FastAPI automáticamente:
# 1. Extrae el token del header (oauth2_scheme)
# 2. Valida el token (obtener_usuario_actual)
# 3. Busca el usuario en la BD (obtener_usuario_actual)
# 4. Inyecta el objeto User en current_user
@app.get("/usuarios/me", response_model=User)
async def obtener_mi_perfil(current_user: CurrentUser):
    # Simplemente retornar el usuario actual
    # La validación ya ocurrió en la dependencia
    return current_user

# Pruebas
# Acceso SIN token
response_sin_token = client.get("/usuarios/me")
assert response_sin_token.status_code == 401
print("✅ Acceso sin token denegado")

# Acceso CON token
response_con_token = client.get("/usuarios/me", headers={"Authorization": f"Bearer {token}"})
assert response_con_token.status_code == 200
user_data = response_con_token.json()
assert user_data["username"] == "admin"
assert user_data["email"] == "admin@example.com"
print("✅ Perfil del usuario obtenido correctamente")
print(f"   Perfil: {user_data}")

print("\n✅ Ejercicio 8 completado")
print("\n💡 Patrón común:")
print("  GET /usuarios/me → Perfil del usuario autenticado")
print("  PUT /usuarios/me → Actualizar perfil")
print("  DELETE /usuarios/me → Eliminar cuenta")

---

## SOLUCIÓN EJERCICIO 9: Manejo de token expirado (401)

In [ ]:
import time

# Crear token que expira en 1 segundo
payload_corto = {
    "sub": "admin",
    "exp": datetime.utcnow() + timedelta(seconds=1)  # Expira en 1 segundo
}
token_corto = jwt.encode(payload_corto, SECRET_KEY, algorithm=ALGORITHM)

# Verificar que funciona ANTES de expirar
response_valido = client.get("/usuarios/me", headers={"Authorization": f"Bearer {token_corto}"})
assert response_valido.status_code == 200
print("✅ Token válido antes de expirar")

# Esperar 2 segundos para que expire
print("⏳ Esperando 2 segundos para que expire...")
time.sleep(2)

# Verificar que el token expirado es rechazado
response_expirado = client.get("/usuarios/me", headers={"Authorization": f"Bearer {token_corto}"})

# Verificación
assert token_corto is not None
assert response_expirado.status_code == 401
print("✅ Token expirado correctamente rechazado")
print(f"   Status: {response_expirado.status_code}")
print(f"   Detail: {response_expirado.json()['detail']}")

print("\n✅ Ejercicio 9 completado")
print("\n💡 Nota: jwt.decode() verifica automáticamente el campo 'exp'")
print("   Si el token expiró, lanza PyJWTError con mensaje 'Signature has expired'")

---

## SOLUCIÓN EJERCICIO 10: Sistema completo (Register + Login + Perfil)

In [ ]:
# Modelo para registro
class UserRegister(BaseModel):
    username: str
    email: str
    password: str  # Llega en texto plano

# Endpoint de registro
@app.post("/register", response_model=User)
async def registrar_usuario(user_data: UserRegister):
    """
    Registra un nuevo usuario:
    1. Valida que el username no existe
    2. Hashea la contraseña (NUNCA guardar en texto plano)
    3. Guarda el usuario en la BD
    4. Retorna el usuario creado (sin la contraseña)
    """
    # 1. Validar que el usuario no existe
    if user_data.username in USERS_DB:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="El usuario ya existe"
        )
    
    # 2. Hashear la contraseña
    # NUNCA guardar passwords en texto plano
    hashed_password = pwd_context.hash(user_data.password)
    
    # 3. Guardar usuario en la base de datos
    USERS_DB[user_data.username] = {
        "username": user_data.username,
        "email": user_data.email,
        "hashed_password": hashed_password  # Solo guardamos el hash
    }
    
    # 4. Retornar usuario creado (SIN la password)
    return User(
        username=user_data.username,
        email=user_data.email
    )

print("✅ Endpoint POST /register definido")

### Prueba del flujo completo: Register → Login → Perfil

In [ ]:
print("🔐 FLUJO COMPLETO: Register → Login → Perfil\n")
print("="*50)

# 1. REGISTRO
print("\n1️⃣ REGISTRO")
response_register = client.post(
    "/register",
    json={"username": "testuser", "email": "test@example.com", "password": "testpass123"}
)
assert response_register.status_code == 200
registered_user = response_register.json()
print(f"   ✅ Usuario registrado: {registered_user['username']}")
print(f"   📧 Email: {registered_user['email']}")
print(f"   🔒 Password hasheada en BD")

# 2. LOGIN
print("\n2️⃣ LOGIN")
response_login = client.post(
    "/token",
    data={"username": "testuser", "password": "testpass123"}
)
assert response_login.status_code == 200
new_token = response_login.json()["access_token"]
print(f"   ✅ Login exitoso")
print(f"   🎫 Token JWT generado: {new_token[:40]}...")
print(f"   ⏰ Válido por 30 minutos")

# 3. ACCESO A PERFIL PROTEGIDO
print("\n3️⃣ ACCESO A RUTA PROTEGIDA")
response_perfil = client.get(
    "/usuarios/me",
    headers={"Authorization": f"Bearer {new_token}"}
)
assert response_perfil.status_code == 200
perfil = response_perfil.json()
print(f"   ✅ Acceso autorizado")
print(f"   👤 Usuario: {perfil['username']}")
print(f"   📧 Email: {perfil['email']}")

# 4. INTENTO DE USUARIO DUPLICADO
print("\n4️⃣ VALIDACIÓN: USUARIO DUPLICADO")
response_duplicate = client.post(
    "/register",
    json={"username": "testuser", "email": "otro@example.com", "password": "otra"}
)
assert response_duplicate.status_code == 400
print(f"   ✅ Usuario duplicado rechazado")
print(f"   ❌ Status: {response_duplicate.status_code}")
print(f"   💬 Detail: {response_duplicate.json()['detail']}")

# 5. INTENTO SIN TOKEN
print("\n5️⃣ VALIDACIÓN: ACCESO SIN TOKEN")
response_sin_auth = client.get("/usuarios/me")
assert response_sin_auth.status_code == 401
print(f"   ✅ Acceso sin token denegado")
print(f"   ❌ Status: {response_sin_auth.status_code}")

print("\n" + "="*50)
print("✅ EJERCICIO 10 COMPLETADO")
print("✅ TODOS LOS EJERCICIOS FINALIZADOS")
print("="*50)

print("\n💡 Conceptos clave practicados:")
print("   1. Hash de passwords con bcrypt")
print("   2. Generación de JWT con PyJWT")
print("   3. OAuth2PasswordBearer para extraer tokens")
print("   4. Dependencias para validar autenticación")
print("   5. Flujo completo: Register → Login → Acceso protegido")

---

## RESUMEN DE BUENAS PRÁCTICAS

### Seguridad
1. ✅ **NUNCA** guardar contraseñas en texto plano
2. ✅ Usar bcrypt (o argon2) para hashear passwords
3. ✅ SECRET_KEY debe estar en variables de entorno
4. ✅ Tokens deben tener fecha de expiración
5. ✅ Validar firma JWT en cada request

### Arquitectura
1. ✅ Usar dependencias reutilizables (`CurrentUser`)
2. ✅ Separar lógica de autenticación en funciones
3. ✅ OAuth2PasswordBearer para extraer tokens
4. ✅ HTTPException 401 para errores de autenticación
5. ✅ response_model para no exponer passwords

### Endpoints estándar
- `POST /register` → Crear usuario
- `POST /token` → Login (retorna JWT)
- `GET /usuarios/me` → Perfil del usuario autenticado
- Rutas protegidas → Usar `current_user: CurrentUser`